In [12]:
import pandas as pd

PROD_RUN = True
cm_features = pd.read_csv('../data/cm_features_v1.0.csv')
cm_features['date'] = pd.to_datetime(cm_features['date'])
# create country_id to ccode mapping
country_id_to_ccode = cm_features[['country_id', 'ccode']].drop_duplicates()
cm_features

,month_id,country_id,gleditsch_ward,ged_sb,ged_ns,ged_os,acled_sb,acled_sb_count,acled_os,ged_sb_tsum_24,...,region23_Northern America,region23_Northern Europe,region23_South America,region23_South-Eastern Asia,region23_Southern Africa,region23_Southern Asia,region23_Southern Europe,region23_Western Africa,region23_Western Asia,region23_Western Europe
0,121,1,110,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,121,2,115,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,121,3,52,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,121,4,101,0,0,12,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,121,7,160,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63203,502,242,510,0,0,1,0,0,2,30,...,0,0,0,0,0,0,0,0,0,0
63204,502,243,600,0,0,0,2,2,0,5,...,0,0,0,0,0,0,0,0,0,0
63205,502,244,435,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
63206,502,245,625,3,3,9,2,1,42,82,...,0,0,0,0,0,0,0,0,0,0


In [17]:
import numpy as np

prediction_years = [2021]
prediction_window = 14
column_name = f'ged_sb_{prediction_window}'
for prediction_year in prediction_years:
    print(f'Prediction year: {prediction_year}')
    features_to_oct = pd.Timestamp(year=prediction_year - 1, month=10, day=1)  # 2021-Oct-01
    cm_features_year = cm_features[cm_features['date'] <= features_to_oct]
    # get last month_id
    last_month_id = cm_features_year['month_id'].max()
    last_month_cm_features = cm_features_year[cm_features_year['month_id'] == last_month_id]
    # create 3 month window based on last_month_cm_features
    two_month_buffer_features = []
    for counter in range(1, 3):
        temp_month = last_month_cm_features.copy()
        temp_month['month_id'] = last_month_id + counter
        temp_month['ged_sb'] = np.nan
        two_month_buffer_features.append(temp_month)
    two_month_buffer_features = pd.concat(two_month_buffer_features)
    # read actuals for this year
    actuals_year = pd.read_parquet(f'../actuals/cm/window=Y{prediction_year}/cm_actuals_{prediction_year}.parquet')
    # add ccode column to actuals_year
    actuals_year = actuals_year.merge(country_id_to_ccode, on='country_id', how='left')
    if not PROD_RUN:
        print("Not prod run, breaking...")
        break
    actuals_year = actuals_year[~actuals_year['ccode'].isnull()]

    _gap_months = two_month_buffer_features['month_id'].unique() - 11 - 3
    test_set_months_min = cm_features_year['month_id'].max() - 11
    test_set_months_max = cm_features_year['month_id'].max()
    print(f"expected empty months because of the gap: {_gap_months}")
    print(f"test set is from {test_set_months_min} to {test_set_months_max}")
    print(f"two month buffer months: {two_month_buffer_features['month_id'].unique()}")

    cm_features_year = pd.concat([cm_features_year, two_month_buffer_features, actuals_year])
    cm_features_year.reset_index(drop=True, inplace=True)

    cm_features_year[column_name] = cm_features_year.groupby('ccode')['ged_sb'].shift(-prediction_window)
    # drop rows with these months: actuals_year['month_id'].unique()
    cm_features_year = cm_features_year[~cm_features_year['month_id'].isin(actuals_year['month_id'].unique())]
    # drop rows with two_month_buffer_features['month_id'].unique()
    cm_features_year = cm_features_year[
        ~cm_features_year['month_id'].isin(two_month_buffer_features['month_id'].unique())]

    month_ids_is_null = cm_features_year[cm_features_year[column_name].isnull()]['month_id'].unique()
    assert all(_gap_months == month_ids_is_null), "Unexpected missing months"
    
    # drop gap months
    cm_features_year = cm_features_year[~cm_features_year['month_id'].isin(_gap_months)]

    # drop country column
    cm_features_year.drop(columns=['country'], inplace=True)
    cm_features_year.to_csv(f'../data/cm_features_v1.0_Y{prediction_year}.csv', index=False)

if PROD_RUN:
    print("All done!")

Prediction year: 2021
expected empty months because of the gap: [477 478]
test set is from 479 to 490
two month buffer months: [491 492]
All done!


In [18]:
cm_features_year['month_id'].unique()
# cm_features_year[cm_features_year["ged_sb_14"].isnull()]

array([121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
       199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
       225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
       238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
       251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
       264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
       277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289,
       290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 30

In [15]:
if not PROD_RUN:
    # For verification of actuals
    # # read data/country_list.csv
    # country_list = pd.read_csv('../data/country_list.csv')
    # country_list
    # # convert country_id to country_name
    # # after inspection, I decide to neglect these countries as they are small and fatalities are 0. Also data coverage is low for these countries (should be verified)
    # actuals_year = actuals_year.merge(country_list[['country_id', 'name']], on='country_id', how='left')
    # actuals_year[actuals_year['ccode'].isnull()][['country_id', 'ccode', 'ged_sb', 'name']].sort_values('ged_sb',
    #                                                                                                     ascending=False)
    # drop rows with ccode is null
    actuals_year = actuals_year[~actuals_year['ccode'].isnull()]

    print(cm_features_year['month_id'].unique())
    print(two_month_buffer_features['month_id'].unique())
    # expected empty month is:
    _gap_months = two_month_buffer_features['month_id'].unique() - 11 - 3
    print(f"expected empty months because of the gap: {_gap_months}")
    test_set_months_min = cm_features_year['month_id'].max() - 11
    test_set_months_max = cm_features_year['month_id'].max()
    print(f"test set should be from {test_set_months_min} to {test_set_months_max}")
    print(f"Actuals months: {actuals_year['month_id'].unique()}")

    cm_features_year = pd.concat([cm_features_year, two_month_buffer_features, actuals_year])
    cm_features_year.reset_index(drop=True, inplace=True)
    # cm_features_year

    column_name = 'ged_sb_14'
    months_ahead = 14
    cm_features_year[column_name] = cm_features_year.groupby('ccode')['ged_sb'].shift(-months_ahead)
    # drop rows with these months: actuals_year['month_id'].unique()
    cm_features_year = cm_features_year[~cm_features_year['month_id'].isin(actuals_year['month_id'].unique())]
    # drop rows with two_month_buffer_features['month_id'].unique()
    cm_features_year = cm_features_year[
        ~cm_features_year['month_id'].isin(two_month_buffer_features['month_id'].unique())]
    cm_features_year

    # get month_id where ged_sb_14 is NaN
    month_ids_is_null = cm_features_year[cm_features_year[column_name].isnull()]
    # get all month_ids is null for months before _gap_months min
    # month_ids_is_null_before = month_ids_is_null[month_ids_is_null['month_id'] < _gap_months.min()]

    month_ids_is_null[['month_id', 'country_id', 'gw_statename', 'ccode', 'ged_sb', column_name]].drop_duplicates()

    cm_features_year[cm_features_year['ccode'] == 530][
        ['month_id', 'country_id', 'ccode', 'country', 'gw_statename', 'ged_sb', 'ged_sb_14']]